In [ ]:

!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 17.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)


In [ ]:
import yt_dlp as youtube_dl
import os
import whisper
from langdetect import detect
import requests
from bs4 import BeautifulSoup
import json


def download_youtube_video(url):
    ydl_opts = {
        'format': 'best',
        'outtmpl': './videos/%(title)s.%(ext)s',
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

# Function to open a file
def startfile(fn):
    os.system('open %s' % fn)

# Function to create and open a txt file
def create_and_open_txt(text, filename):
    # Create and write the text to a txt file
    with open(filename, "w") as file:
        file.write(text)
    startfile(filename)




url = input("Enter the Url: ")

#get youtube video title
r = requests.get(url)
soup = BeautifulSoup(r.text)

link = soup.find_all(name="title")[0]
title = str(link)
title = title.replace("<title>","")
title = title.replace("</title>","")

title = title[0:-10]
title2 = ".mp4"
newtitle = title+title2
print(newtitle)

#get youtube video to download
video = download_youtube_video(url)



#convert the vid in to audio for transcribe
import moviepy.editor as mp
path = "/content/videos/"+newtitle
# Load the video
video = mp.VideoFileClip(path)

# Extract the audio from the video and save it
audio_file = video.audio
new2 = title+".wav"
audio_file.write_audiofile(new2)

#audio to text transcribe
model = whisper.load_model("base")
result = model.transcribe(new2)
transcribed_text = result["text"]
print(transcribed_text)

# Detect the language
language = detect(transcribed_text)
print(f"Detected language: {language}")

# Create and open a txt file with the text and save it
create_and_open_txt(transcribed_text, f"{title}_{language}.csv")



existing_data = []

with open("sample.json") as json_file:
    existing_data = json.load(json_file)

existing_data.append({
    "Date": "21/05/24",
    "Reference Url": url,
    "Hook": transcribed_text[0:200],
    "Build Up": transcribed_text[0:400],
    "Body": transcribed_text,
    "CTA": transcribed_text[-150:]
})

with open('sample.json', 'w') as json_file:
    json.dump(existing_data, json_file,
                        indent=4,
                        separators=(',',': '))


